## Utiliziamo LLM per risolvere problemi di Sentiment Analysis
- Utilizza il dataset 'SetFit/tweet_sentiment_extraction' presente sull'hub di HuggingFace e che ti viene fornito nelle prime celle di questo notebook (utilizza un sample di alcune tweet)
- Carica in memoria il modello microsoft/Phi-3-mini-4k-instruct da utilizzare per generare del testo (é consigliabile va bene anche caricarlo in memoria in formato 8 o 4bit)
- Scrivi delle funzioni per generare il prompt nel formato corretto specifico utilizzato dal modello. Il prompt deve essere parametrizzato e contenere informazioni sul task da svolgere e il testo su cui andare a determinare il sentiment.
- Prova a confrontare prompt con approcci diversi (Zero-Shot, Few-Shot, ...)
- Formatta l'output del modello in modo da restituire solamente il sentiment del testo
- Confronta le performance di classificazione ottenuta generando il sentiment con i diversi approcci di Prompt Engineering


<font color='red'>Va BENISSIMO fare copia-incolla dai notebook precedenti: <br>
&nbsp;&nbsp;&nbsp;&nbsp; → non devi imparare cose a memoria, devi capire come funzionano e sapere dove copiare, cosa modifiare e come! <br>
 Riuscirai a utilizzare dei modelli LLM generativi per risolvere dei task di NLP?
</font>

In [ ]:
!pip install flash_attn==2.5.8
!pip install torch==2.3.1
!pip install accelerate==0.31.0
!pip install transformers==4.41.2
!pip -q install bitsandbytes accelerate xformers einops
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 11.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cu

In [ ]:
import re
from sklearn.datasets import fetch_20newsgroups
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.metrics import classification_report
from datasets import load_dataset


In [ ]:
device = torch.device("cuda")

In [ ]:
ds = load_dataset("SetFit/tweet_sentiment_extraction")
ds_texts = ds['train']['text']
ds_targets = ds['train']['label_text']

Repo card metadata block was not found. Setting CardData to empty.


In [ ]:
ds_texts[3]

' what interview! leave me alone'

In [ ]:
ds_targets[3]

'negative'

In [ ]:
possible_sentiments = list(set(ds_targets))

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct", trust_remote_code=True,
                                          torch_dtype="auto",
                                          load_in_4bit=True)
model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-mini-4k-instruct", trust_remote_code=True,
                                             torch_dtype="auto",
                                          load_in_4bit=True)

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/16.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [ ]:
def create_prompt(text, possible_sentiments = possible_sentiments):
  text = text[:10000]
  prompt = f"""<|system|>
  You are a helpful assistant gives a sentiment to a twitter text.<|end|>
  <|user|>
  Please give a sentiment to the following text. The possible values for the sentiment are:
  {possible_sentiments}
  Return only your best guess of the sentiment of the sentence provided below.

  Text:
  {text}

  Class: <|end|>
  <|assistant|>"""

  return prompt

In [ ]:
prompt = create_prompt(ds_texts[3])
print(prompt)

<|system|>
  You are a helpful assistant gives a sentiment to a twitter text.<|end|>
  <|user|>
  Please give a sentiment to the following text. The possible values for the sentiment are:
  ['positive', 'negative', 'neutral']
  Return only your best guess of the sentiment of the sentence provided below.

  Text:
   what interview! leave me alone

  Class: <|end|>
  <|assistant|>


In [ ]:
def generate(input_text, temperature=0.1, do_sample=True, max_new_tokens=100):
    inputs = tokenizer(input_text, return_tensors="pt", add_special_tokens=False)
    model_inputs = inputs.to(device)
    outputs = model.generate(**model_inputs,
                             max_new_tokens=max_new_tokens,
                             temperature=temperature,
                             do_sample=do_sample)

    tokenized_output = tokenizer.batch_decode(outputs[:, model_inputs["input_ids"].shape[1]:])[0]
    return tokenized_output

In [ ]:
def clean_output(original_output):
    cleaned_string = re.sub(re.escape('<|end|>'), "", original_output)
    pattern = r"'([^']*)'"
    matched = re.search(pattern, cleaned_string)
    if matched:
        return matched.group(1)
    else:
        return cleaned_string


In [ ]:
prompt

"<|system|>\n  You are a helpful assistant gives a sentiment to a twitter text.<|end|>\n  <|user|>\n  Please give a sentiment to the following text. The possible values for the sentiment are:\n  ['positive', 'negative', 'neutral']\n  Return only your best guess of the sentiment of the sentence provided below.\n\n  Text:\n   what interview! leave me alone\n\n  Class: <|end|>\n  <|assistant|>"

In [ ]:
output = generate(prompt, temperature=0.0, do_sample=False, max_new_tokens=15)
cleaned_output = clean_output(output)
print(cleaned_output)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


negative


In [ ]:
output_list, target_list = [], []
c = 0
with torch.no_grad():
  for el in ds['train']:
    text = el['text']
    target_name = el['label_text']
    prompt = create_prompt(text, possible_sentiments=possible_sentiments)
    output = generate(prompt, temperature=0.0, do_sample=False, max_new_tokens=20)
    cleaned_output = clean_output(output)
    print(cleaned_output)
    output_list.append(cleaned_output)
    target_list.append(target_name)
    c+=1
    if c==100:
      break

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


negative
negative
negative
negative
negative
positive
positive
neutral
neutral
positive
negative
positive
negative
negative
neutral
negative
negative
negative
positive
negative
positive
positive
neutral
negative
neutral
positive
negative
negative
positive
negative
neutral
positive
negative
positive
neutral
negative
neutral
positive
negative
positive
negative
positive
negative
negative
neutral
negative
negative
neutral
negative
negative
negative
negative
neutral
negative
neutral
neutral
negative
neutral
negative
positive
negative
positive
negative
positive
negative
neutral
positive
negative
neutral
neutral
negative
neutral
positive
neutral
negative
neutral
positive
negative
negative
negative
positive
negative
negative
negative
negative
positive
neutral
negative
negative
neutral
positive
neutral
neutral
negative
positive
positive
negative
negative
neutral
negative


In [ ]:
print(classification_report(target_list, output_list))

              precision    recall  f1-score   support

    negative       0.60      0.91      0.72        33
     neutral       0.72      0.42      0.53        43
    positive       0.64      0.67      0.65        24

    accuracy                           0.64       100
   macro avg       0.65      0.66      0.64       100
weighted avg       0.66      0.64      0.62       100



In [ ]:
dict_targets = {}
for el in ds['test']:
  text = el['text']
  target_name = el['label_text']
  if target_name not in dict_targets.keys():
    dict_targets[target_name] = text
  if len(dict_targets) == len(possible_sentiments):
    break

In [ ]:
def create_few_shot_prompt(text, text_target_dict, possible_sentiments=possible_sentiments):
  few_shot_examples = [f"Text: {value}\nClass: {key}" for key,value in text_target_dict.items()]
  few_shot_examples = "\n".join(few_shot_examples)
  prompt = f"""<|system|>
  You are a helpful assistant gives a sentiment to a twitter text.<|end|>
  <|user|>
  Please give a sentiment to the following text. The possible values for the sentiment are:
  {possible_sentiments}
  Return only your best guess of the sentiment of the sentence provided below.

  Here are some examples to follow for guidelines:
  {few_shot_examples}

  Text:
  {text}

  Class: <|end|>
  <|assistant|>"""

  return prompt

In [ ]:
print(create_few_shot_prompt(text, dict_targets))

<|system|>
  You are a helpful assistant gives a sentiment to a twitter text.<|end|>
  <|user|>
  Please give a sentiment to the following text. The possible values for the sentiment are:
  ['positive', 'negative', 'neutral']
  Return only your best guess of the sentiment of the sentence provided below.

  Here are some examples to follow for guidelines:
  Text: Last session of the day  http://twitpic.com/67ezh
Class: neutral
Text:  Shanghai is also really exciting (precisely -- skyscrapers galore). Good tweeps in China:  (SH)  (BJ).
Class: positive
Text: Recession hit Veronique Branquinho, she has to quit her company, such a shame!
Class: negative

  Text:
  Recession hit Veronique Branquinho, she has to quit her company, such a shame!

  Class: <|end|>
  <|assistant|>


In [ ]:
output_list, target_list = [], []
c = 0
with torch.no_grad():
  for el in ds['train']:
    text = el['text']
    target_name = el['label_text']
    prompt = create_few_shot_prompt(text, dict_targets, possible_sentiments=possible_sentiments)
    output = generate(prompt, temperature=0.0, do_sample=False, max_new_tokens=20)
    cleaned_output = clean_output(output)
    print(cleaned_output)
    output_list.append(cleaned_output)
    target_list.append(target_name)
    c+=1
    if c==100:
      break

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


negative
negative
negative
negative
negative
neutral
positive
negative
neutral
positive
negative
positive
negative
negative
neutral
negative
negative
negative
negative
negative
negative
positive
neutral
negative
negative
positive
negative
negative
neutral
negative
neutral
negative
negative
positive
neutral
negative
neutral
neutral
negative
positive
negative
positive
negative
negative
neutral
negative
negative
neutral
negative
negative
negative
negative
neutral
negative
neutral
neutral
negative
negative
negative
neutral
negative
neutral
negative
positive
negative
neutral
positive
negative
negative
neutral
negative
neutral
positive
negative
negative
neutral
negative
negative
negative
negative
positive
negative
negative
negative
negative
positive
neutral
negative
negative
neutral
positive
negative
neutral
negative
neutral
positive
negative
negative
neutral
negative


In [ ]:
print(classification_report(target_list, output_list))

              precision    recall  f1-score   support

    negative       0.53      0.97      0.69        33
     neutral       0.72      0.42      0.53        43
    positive       0.93      0.58      0.72        24

    accuracy                           0.64       100
   macro avg       0.73      0.66      0.65       100
weighted avg       0.71      0.64      0.63       100

